In [7]:
pip install patsy

   ---------------------------------------- 0.0/232.9 kB ? eta -:--:--
   - -------------------------------------- 10.2/232.9 kB ? eta -:--:--
   - -------------------------------------- 10.2/232.9 kB ? eta -:--:--
   ----- --------------------------------- 30.7/232.9 kB 262.6 kB/s eta 0:00:01
   -------------------------- ------------- 153.6/232.9 kB 1.0 MB/s eta 0:00:01
   -------------------------------------- - 225.3/232.9 kB 1.4 MB/s eta 0:00:01
   -------------------------------------- 232.9/232.9 kB 951.6 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\Estefany\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import patsy
from sklearn.preprocessing import StandardScaler
import warnings
warnings.simplefilter('ignore')


In [2]:
data = pd.read_csv('../data/raw/wage2015_subsample_inference.csv') # Loading the data
list(data.columns)

['wage',
 'lwage',
 'sex',
 'shs',
 'hsg',
 'scl',
 'clg',
 'ad',
 'mw',
 'so',
 'we',
 'ne',
 'exp1',
 'exp2',
 'exp3',
 'exp4',
 'occ',
 'occ2',
 'ind',
 'ind2']

# Processing the data

In [3]:
X = data.drop(['wage', 'lwage'], axis=1)
y = data[['lwage']]

The extra-flexible model

In [4]:
X_flexible = patsy.dmatrix('0 + sex + (exp1+exp2+exp3+exp4+hsg+scl+clg+ad+so+we+ne+C(occ2)+C(ind2))**2',
                      X, return_type='dataframe')

Generalize
* the array for the outcome variable Y and normalize it
* the array for the predictors X

In [5]:
y = y.to_numpy()
X_flexible = X_flexible.to_numpy()

# Lasso Cross-Validation

## Creating Procedure

`Log_grid` : Function that outputs a log-spaced grid.

--> The input arguments should be the natural logarithms of the lower and upper bounds of the grid, as well as the natural logarithm of the spacing between each grid element. 

--> The output be the log-spaced grid, meaning that if we take the natural logarithm of each input in the grid, they will be evenly spaced. 

In [6]:
def log_grid(lower:int, upper:int, log_step:int):  
    log_grid = np.linspace(lower,upper,int(1/log_step))
    return np.exp(log_grid)

`k_folds` : Function to generate k folds.  It output a list of k arrays of booleans (the same length as the number of rows in the input array, and when they are all summed together they should add up to an array of all true values)

In [7]:
def k_folds(data:np.ndarray, k:int = 5):

    
    module = data.shape[0]%k
    floor = data.shape[0]//k 

    if module == 0:
        
        trues = np.repeat(1, floor).reshape(-1,1)

        split_matrix = np.kron(np.eye(k), trues)
        
    else:
        
        trues_g1 = np.repeat(1,floor+1).reshape(-1,1)

        split_matrix_g1 = np.kron(np.eye(module), trues_g1)

        trues_g2 = np.repeat(1,floor).reshape(-1,1)

        split_matrix_g2 = np.kron(np.eye(k-module), trues_g2)
        
        split_matrix = np.block([[split_matrix_g1, np.zeros((split_matrix_g1.shape[0],split_matrix_g2.shape[1]))],
                                     [np.zeros((split_matrix_g2.shape[0],split_matrix_g1.shape[1])), split_matrix_g2]])
        
    sm_bool = split_matrix == 1
    
    splits = [sm_bool[:,x] for x in range(k)]
        
    return splits

`Optimal Lambda` : Function to find the value of λ that minimizes the testing mean square error across folds.

In [8]:
def optimal_lambda(Y:np.ndarray, X:np.ndarray, lambda_bounds:tuple, k:int=5, *, niter:int = 100):
    
    from sklearn.linear_model import Lasso
    
    Y = Y.squeeze()
    
    if len(X.shape) == 1:
        
        X = X.reshape(-1,1)
        
    folds = k_folds(X,k)
    
    all_lambdas = log_grid(lambda_bounds[0],lambda_bounds[1],1/niter)
        
    all_mse = np.zeros(niter)

    for l in all_lambdas:
        
        split_pes = np.zeros(k)
        
        for i in range(k):
            
            X_train, X_test = X[~folds[i]],  X[folds[i]]
            y_train, y_test = Y[~folds[i]],  Y[folds[i]]

            
            model = Lasso(alpha=l).fit(X_train, y_train)
            
            predict = model.predict(X_test)
            
            pe = np.sum((y_test-predict)**2)
            
            split_pes[i] = pe
            
        all_mse[all_lambdas == l] = np.mean(split_pes)

    selected = np.where(all_mse == np.min(all_mse))

    optimal_lambda = all_lambdas[selected][0]

    optimal_model = Lasso(alpha=optimal_lambda).fit(X,Y)
    
    optimal_coef = np.hstack((optimal_model.intercept_,optimal_model.coef_))
    
    output = {'optimal_lambda':optimal_lambda, 'optimal_coef':optimal_coef,
              'all_lambdas': all_lambdas, 'all_mse':all_mse}
    
    return output

`predict_model` : Function for predicting the outcome variable through model estimated with the optimal lambda. 

*Inputs*

optimal_model: A dictionary with the values outputed by the function defined for the previous point.

X: an array of predictors.

*Output*

An array of predicted values.

In [9]:
def predict_model(optimal_model:dict, X:np.ndarray):

    intercept = np.ones((X.shape[0],1))
    Z = np.hstack((intercept,X))
    return Z@optimal_model['optimal_coef'].reshape(-1,1)

## Applying the Lasso Cross-Validation Procedure

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_flexible_train, X_flexible_test, y_train, y_test = train_test_split(X_flexible, y, test_size= 0.2) # train and test

Simple OLS model fitting with the training sample

In [12]:
from sklearn.linear_model import LinearRegression
model_ls = LinearRegression()
model_ls.fit(X_flexible_train,y_train)

LinearRegression()

### The optimal lambda

The lower and upper bounds of the grid between -7 and 7 respectively. 

In [14]:
model_lasso = optimal_lambda(y_train,X_flexible_train, (-7,7)) #using our `optimal_lambda` function and the training sample.

In [15]:
print(model_lasso['all_mse'])

[199.09459528 198.56139062 198.14129732 197.87863835 197.83385216
 197.95261891 198.2061413  198.54794259 199.06794861 199.80756929
 200.67743304 201.79027107 202.6949363  203.55987112 204.47815748
 205.54546146 206.96876689 208.652668   209.94871188 211.08409079
 211.83395632 212.23405169 212.5920356  213.13867248 213.9482346
 214.76689968 215.75613626 216.90330411 218.31895613 219.81348636
 221.13575556 222.5918252  224.40293347 226.38249951 228.34936614
 230.49691912 232.82077514 235.22839831 237.61309804 240.21086707
 243.2199789  245.84811153 247.65271778 250.0892911  253.95931177
 258.73870855 261.73345948 264.98259726 267.29205251 268.3058013
 269.08446416 269.59447243 270.06731718 270.10862342 270.11659871
 270.12772964 270.14312846 270.16416816 270.19315006 270.23270469
 270.28639825 270.35898179 270.4568677  270.58857584 270.76540714
 271.0024847  271.30535365 271.50529782 271.60766364 271.61819732
 271.63239971 271.65150428 271.67715289 271.70252312 271.72183335
 271.7513000

In [16]:
print(model_lasso['optimal_lambda'])

0.0016054624479407073


In [17]:
print(model_lasso['optimal_coef'])

[ 2.73204353e+00  1.98831545e-01  1.37838089e-01  2.09772662e-01
  2.47262979e-02  0.00000000e+00 -0.00000000e+00  0.00000000e+00
 -9.27839937e-02  0.00000000e+00  4.99128249e-02 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
  0.00000000e+00 -8.46105635e-02 -0.00000000e+00  0.00000000e+00
 -0.00000000e+00  0.00000000e+00 -0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00 -2.17768125e-02 -0.00000000e+00  0.00000000e+00
  5.03402177e-02  0.00000000e+00  9.06073121e-03 -0.00000000e+00
 -0.00000000e+00 -1.03324281e-01 -9.13700423e-03 -0.00000000e+00
 -2.00422725e-01 -0.00000000e+00  0.00000000e+00 -0.00000000e+00
  0.00000000e+00 -0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -0.00000000e+00  0.00000000e+00
 -0.00000000e+00  0.00000

HDM for python (hdmpy) to estimate the model using the theoretically optimal penalty parameter.

the [package documentation](https://arxiv.org/pdf/1608.00354)

### The predictive capability of each model (OLS, Lasso and RLasso)

Report the testing $MSE$ and $R^2$ 

In [23]:
# OLS 

y_predict_ols = model_ls.predict(X_flexible_test)
MSE_ols = np.mean((y_test-y_predict_ols)**2)
R2_test_ols = 1-MSE_ols/np.var(y_test)
print(R2_test_ols)

0.18585163126029125


In [24]:
# Lasso CV

y_predict_lasso = predict_model(model_lasso, X_flexible_test)
MSE_lasso = np.mean((y_test-y_predict_lasso)**2)
R2_test_lasso = 1-MSE_lasso/np.var(y_test)
print(R2_test_lasso)

0.2865806354879117
